In [1]:
import warnings
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
warnings.filterwarnings('ignore')

In [3]:
cancers = ['OV', 'CRC', 'BRCA']
mutations = ['missense', 'truncating', 'synonymous']

## Concat OV files from different centers

In [4]:
mut = 'missense'

In [5]:
for mut in mutations:   
    jhu = pd.read_excel('../data/Pro_regression_retro_cptac/Table.DNA.PRO.JHU.regression.linearLIMMA.ProVsMut.OV.' +
                      mut + '.xlsx')
    pnnl = pd.read_excel('../data/Pro_regression_retro_cptac/Table.DNA.PRO.PNNL.regression.linearLIMMA.ProVsMut.OV.' +
                      mut + '.xlsx')
    df = pd.concat([jhu, pnnl], axis=0)
    df.to_excel('../data/Pro_regression_retro_cptac/Table.DNA.PRO.regression.linearLIMMA.ProVsMut.OV.' + mut + '.xlsx')

## Overlap prosective and retrosepctive data

In [6]:
dat = pd.DataFrame()

for cancer in cancers:
    for mut in mutations:
        pros = pd.read_excel('../data/DNA_RNA_regression/Table.DNA.RNA.regression.linearLIMMA.RNAVsMut.' + 
                             cancer + '.' + mut + '.xlsx')
        retro = pd.read_excel('../data/RNA_regression_retro_cptac/Table.DNA.RNA.regression.linearLIMMA.RNAVsMut.' + 
                              cancer + '.' + mut + '.xlsx')
#         pros = pd.read_excel('../data/DNA_PRO_regression/Table.DNA.PRO.regression.linearLIMMA.ProVsMut.' + 
#                              cancer + '.' + mut + '.xlsx')
#         retro = pd.read_excel('../data/Pro_regression_retro_cptac/Table.DNA.PRO.regression.linearLIMMA.ProVsMut.' + 
#                               cancer + '.' + mut + '.xlsx')
        sig_pros = pros[pros['P.value'] < 0.05]
        sig_pros = sig_pros[['Gene', 'logFC', 'FDR']]
        sig_retro = retro[retro['P.value'] < 0.05]
        sig_retro = sig_retro[['Gene', 'logFC', 'FDR']]
        df = sig_pros.merge(sig_retro, on='Gene')
        df.columns = ['Gene', 'logFC_pros', 'FDR_pros', 'logFC_retro', 'FDR_retro']
        df['cancer'] = cancer
        df['mutation'] = mut
        df.to_excel('../data/overlap_with_retro/eQTLs/' + cancer + mut + 'p_value.05_overlap_eQTLs.xlsx')
#         df.to_excel('../data/overlap_with_retro/pQTLs/' + cancer + mut + 'p_value.05_overlap_pQTLs.xlsx')
        dat = pd.concat([dat, df], axis=0)

dat.to_excel('../data/overlap_with_retro/eQTLs/overlap_eQTLs.xlsx')
# dat.to_excel('../data/overlap_with_retro/pQTLs/overlap_pQTLs.xlsx')

In [ ]:
overlap = pd.read_excel('../data/overlap_with_retro/eQTLs/overlap_eQTLs.xlsx')
# overlap = pd.read_excel('../data/overlap_with_retro/pQTLs/overlap_pQTLs.xlsx')
overlap = overlap.drop(['Unnamed: 0', '...1'], axis=1).reset_index(drop=True)
overlap.columns = ['Gene', 'logFC.pros', 'FDR.pros', 'logFC.retro', 'FDR.retro', 'mutation', 'cancer']
overlap.to_excel('../data/results/overlap_eQTLs.xlsx')
# overlap.to_excel('../data/results/overlap_pQTLs.xlsx')
overlap.head()

In [ ]:
validated_pQTLs = pd.read_excel('../data/results/overlap_with_retro_pQTLs.xlsx', index_col=[0])
validated_concordant = concordant.merge(validated_pQTLs, on=['Gene', 'cancer', 'mutation'])
validated_concordant.to_excel('../data/results/validated_concordant.xlsx')